# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194
 3.299533727885655
 2.803360380906535
 2.9391619220655967
 2.70805020110221
 2.9391619220655967
 3.077312260546414
 ⋮
 3.1986731175506815
 3.139832617527748
 2.9806186357439426
 2.9069010598473755
 3.054001181677967
 2.8622008809294686
 2.8213788864092133
 3.109060958860994
 3.0252910757955354
 3.173878458937465
 3.091042453358316
 2.4765384001174837

In [4]:
indecies = MLDataUtils.shuffleobs(collect(1: nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8)

train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,3.69311,0.0,18.1,0,0.713,6.376,88.4,2.5671,24,666
2,3.47428,0.0,18.1,1,0.718,8.78,82.9,1.9047,24,666
3,0.11432,0.0,8.56,0,0.52,6.781,71.3,2.8561,5,384
4,9.91655,0.0,18.1,0,0.693,5.852,77.8,1.5004,24,666
5,9.18702,0.0,18.1,0,0.7,5.536,100.0,1.5804,24,666
6,1.15172,0.0,8.14,0,0.538,5.701,95.0,3.7872,4,307
7,0.65665,20.0,3.97,0,0.647,6.842,100.0,2.0107,5,264
8,0.84054,0.0,8.14,0,0.538,5.599,85.7,4.4546,4,307
9,0.15038,0.0,25.65,0,0.581,5.856,97.0,1.9444,2,188


In [5]:
model = GLM.lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.17515      0.23434       17.8166     <1e-51   3.71443       4.63586
Crim         -0.0102378    0.00138139    -7.41125    <1e-12  -0.0129537    -0.00752203
Chas          0.127561     0.0435901      2.92638    0.0036   0.0418631     0.21326
NOx          -0.745822     0.167916      -4.44163    <1e-4   -1.07595      -0.415698
Rm            0.0939744    0.0181086      5.18948    <1e-6    0.0583727     0.129576
Dis          -0.047563     0.00759436    -6.2

In [6]:
exp.(predict(model, test))

101-element Array{Float64,1}:
 18.765912962959312
 35.65347697634357
 24.283315301600307
 11.240991163543864
 13.401622210275018
 15.104625283545671
 35.03588106015688
 15.0247536037511
 14.98412923454637
 26.897804962357114
 21.333888712311627
 17.243581018665694
 16.43135507901442
  ⋮
 34.86972299806809
  8.730244651898197
 24.013492321088336
 28.848993282671174
 17.460817316076213
 27.099091198658563
 18.74424737787501
 15.64418507010648
 13.176331440258789
 17.17639903346013
 30.23165188541801
 16.313650556217755

In [7]:
r2(model)

0.7863628799155927

In [8]:
adjr2(model)

0.7809406179845164